[1. 鸢尾花数据读取](#1)<br>
[2. 建立KNN模型](#2)<br>
&emsp;&emsp;[2.1 KNN模型简介](#2.1)<br>
&emsp;&emsp;[2.2 KNN模型的实现](#2.2)<br>
[3. 模型的训练与评估](#3)<br>
&emsp;&emsp;[3.1 数据集划分](#3.1)<br>
&emsp;&emsp;[3.2 模型评估](#3.2)<br>
[4. 总结](#4)<br>

<div id="1"></div>
# 1. 鸢尾花数据读取

本例中我们用到了鸢尾花（Iris）数据集，这是机器学习和统计学中一个经典的数据集，由R.A. Fisher于1936年收集整理的。其中包含3种植物种类，分别是山鸢尾（setosa）变色鸢尾（versicolor）和维吉尼亚鸢尾（virginica），每类50个样本，共150个样本。

该数据集包含4个特征变量，1个类别变量。Iris每个样本都包含了4个特征：花萼长度，花萼宽度，花瓣长度，花瓣宽度，以及1个类别变量：种类，如下表所示：


|变量名|变量解释|数据类型|
|--|--|--|
|sepal_length|	花萼长度（单位cm）|	numeric|
|sepal_width|	花萼宽度（单位cm）|	numeric|
|petal_length|	花瓣长度（单位cm）|	numeric|
|petal_width|	花瓣宽度（单位cm）|	numeric|
|species|	种类	|categorical|

本篇案例使用Java语言完成这个简单的机器学习应用——对鸢尾花进行分类的任务，并构建KNN模型对新鸢尾花的品种进行预测。

首先，我们定义ArrayList型变量`X`和`y`用于存储数据：

In [1]:
//导入java的常用类
import java.io.*;
import java.util.*;

ArrayList<double[]> X = new ArrayList<>();//X用于存储数据特征变量的取值
ArrayList<String> y = new ArrayList<>();//y用于存储数据类别变量的取值

使用`BufferedReader`读取数据，将特征变量的取值和类别变量的取值分别储存在列表变量`X`和`y`中：

In [2]:
File file = new File("./input/Iris.csv");
BufferedReader readFile = new BufferedReader(new FileReader(file));
String line;
while ((line = readFile.readLine()) != null) {
    String[] split = line.split(",");
    double[] feature = new double[split.length - 1];
    for (int i = 0; i < split.length - 1; i++){
        feature[i] = Double.parseDouble(split[i]);//将字符串变为数值型
        }
    X.add(feature);
    y.add(split[feature.length]);
}
readFile.close();

查看数据集大小：

In [3]:
System.out.print("鸢尾花数据集大小为：" + X.size());

鸢尾花数据集大小为：150

可见数据集一共有150条数据，下面具体的观察数据集的前五行内容：

In [4]:
for (double[] d : X.subList(0,5)) {
    System.out.println(Arrays.toString(d));
    }

[5.1, 3.5, 1.4, 0.2]
[4.9, 3.0, 1.4, 0.2]
[4.7, 3.2, 1.3, 0.2]
[4.6, 3.1, 1.5, 0.2]
[5.0, 3.6, 1.4, 0.2]


每一行数据分别表示花萼长宽和花瓣长宽的取值。

In [5]:
for (String s : y.subList(0,5)) {
    System.out.println(s);
    }

Iris-setosa
Iris-setosa
Iris-setosa
Iris-setosa
Iris-setosa


前五行都是山鸢尾花的数据。由此，得到了上面这样的数据形式，而在后面建立模型时，这种数据形式使得对于参数的处理更加方便。

<div id="2"></div>
# 2. 建立KNN模型

<div id="2.1"></div>
##  2.1 KNN模型简介

$k$近邻算法(k-nearest neighbor)，又称KNN算法，是一种基本的用于分类与回归的机器学习方法。它的基本思想很容易理解，对于分类学习任务，构建此模型只需要保存训练集即可。当要对一个新的数据点做出预测时，算法会在训练集中寻找与这个新数据点距离最近的数据点，然后将找到的数据点的标签赋值给这个新数据点。

$k$近邻算法中$k$的含义是，我们可以考虑训练集中与新数据点最近的任意$k$个“邻居”（比如说，距离最近的3个或5个“邻居”），而不是只考虑最近的那一个。然后，我们可以用这些“邻居”中出现频率最多的类别做出预测。

因此，我们可以通过如下步骤实现KNN算法：

- 1）计算测试数据与各个训练数据之间的距离（这里我们使用欧式距离）；

- 2）按照距离的递增关系进行排序；

- 3）选取距离最近的前$k$个点；

- 4）确定前$k$个点所在类别的出现频率；

- 5）返回前$k$个点中出现频率最高的类别作为测试数据的预测分类。


<div id="2.2"></div>
##  2.2 KNN模型的实现

为使得实现KNN算法的代码更有可读性，我们需要抽象出一些类和方法，首先，我们定义可以计算两条数据之间欧几里得距离的方法`getDistance`。在欧几里得空间中，数据点$x=(x_1,x_2,\ldots,x_n)$和$y=(y_1,y_2,\ldots,y_n)$之间的欧氏距离为：$$d(x,y)=\sqrt{(x_1-y_1)^2 +(x_2-y_2)^2+\ldots+(x_n-y_n)^2}$$

In [6]:
//getDistance方法实现KNN算法的第1步

public double getDistance( double[] features1,  double[] features2) {
    double sum = 0;
    for (int i = 0; i < features1.length; i++){
        sum += Math.pow(features1[i] - features2[i], 2);
    }
    return Math.sqrt(sum);
}

由于算法中包含着对数据的存储和排序操作，因此我们创建新的类`Neighbors`，它用于存储计算出的测试数据与各个训练数据之间的距离，以及训练数据对应的标签信息。同时，为了完成排序操作，这个类还需要实现`Comparable`接口并重写`compareTo`方法，用于比较测试数据与各个训练数据之间距离的远近：

In [7]:
public class Neighbors implements Comparable<Neighbors>{
    double dist;//距离
    String label;//类别
    //构造方法
    public Neighbors(){};
    public Neighbors(double dist, String label){
        this.dist=dist;
        this.label=label;
    }
    //返回测试数据与训练数据的距离
    public double getDist() {
        return dist;
    }
    //返回对应训练数据的类别
    public String getLabel() {
        return label;
    }

    @Override //重写toString方法，可以输出距离和标签信息
    public String toString(){
        return  "距离:" + dist + "; " + "类别:"  + label;
    }

    //实现基于距离进行排序的compareTO方法
    public int compareTo(Neighbors obj) {
        if(this.dist < obj.dist){
            return -1;
        }
        else if(this.dist > obj.dist){
            return 1;
        }
        else{
            return 0;
        }
    }
}

当完成KNN算法的第2步时，这些`Neighbors`类型的数据点按照距离远近排好序并储存在一个ArrayList中。为了统计前$k$个“邻居”点类别出现的频率，并将出现频率最高的类别作为测试数据的预测分类，还需要一个方法`labelPredict`去实现这个功能：

In [8]:
/**
* labelPredict方法实现KNN算法的第3，4，5步
* 方法传入的参数是保存了Neighbors类型数据的ArrayList和表示取前k个数据点的k值
*/
public String labelPredict(ArrayList<Neighbors> neighbors, int k){
    int max = 0;
    String pclass ="";//用于存放最后预测出的类别
    Map<String,Integer> labelfrequency = new HashMap<String,Integer>();
    
    //统计前k个“邻居”类别的频率
    for (int i = 0;i < k; i++){
        Neighbors data = neighbors.get(i);
        int count = 1;
        if (labelfrequency.containsKey(data.getLabel())){
            count = labelfrequency.get(data.getLabel()) + 1;
        }
        labelfrequency.put(data.getLabel(), count);        
    }
    
    //提取出现频率最高的类别
    for(String key : labelfrequency.keySet()){
        if(max < labelfrequency.get(key)){
            max = labelfrequency.get(key);
            pclass = key;
        }
    }      
    return pclass;
}

抽象出上面的类和方法之后，下面实现KNN算法，并将其包装成`KNN`方法。方法`KNN`传入的参数为ArrayList型的训练集里面的数据和标签，以及测试集里面的数据和`k`值，返回测试集通过预测得到的标签列表。

In [9]:
public ArrayList<String> KNN(ArrayList<double[]> X_train, ArrayList<String> y_train, ArrayList<double[]> X_test, int k){
    ArrayList<String> y_prediction = new ArrayList<>();//用于储存测试集预测出的标签
    Iterator<double[]> test_itr = X_test.iterator();//通过迭代器遍历测试集
    while (test_itr.hasNext()) {
        double testdata[] = test_itr.next();
        
        //通过迭代器遍历训练集，寻找与某条测试数据点相邻的训练数据点
        Iterator<double[]> train_itr = X_train.iterator();
        int noOfobject = 0;//用于计数
        ArrayList<Neighbors> neighbors = new ArrayList<>();//用于储存"邻居"数据点
        while (train_itr.hasNext()) {
            double traindata[] = train_itr.next();
            double dist = 0;
            dist = getDistance(traindata, testdata);//计算距离
            String trainLabel = y_train.get(noOfobject);//返回某个训练数据点对应的标签
            Neighbors distAndLabel = new Neighbors(dist, trainLabel);//储存这个训练数据点的距离和标签
            neighbors.add(distAndLabel);
            Collections.sort(neighbors);//将这些训练数据点按距离排序
            noOfobject++;
            }
        y_prediction.add(labelPredict(neighbors, k));//返回测试集预测类别列表
    }   
    return y_prediction;
}

<div id="3"></div>
# 3. 模型的训练与评估

<div id="3.1"></div>
## 3.1 数据集划分

下面我们要使用第2节构建的KNN模型完成对鸢尾花进行分类的任务。首先，我们需要将数据集划分为训练集和测试集，由于数据集数据比较整齐，我们要将数据集里面的数据次序随机打乱，然后再划分数据集。

定义用于打乱数据次序的方法`shuffling`:

In [10]:
public void shuffling (ArrayList<double[]> list1, ArrayList<String> list2){
    for(int i = list1.size()-1; i > 0; i--){
        Random rnd = new Random();
        rnd.setSeed(0);//用于设置随机种子。
        int j = rnd.nextInt(i+1); ;
        double[] temp1 = list1.get(i);
        list1.set(i,list1.get(j));
        list1.set(j, temp1);
        
        String temp2 = list2.get(i);
        list2.set(i,list2.get(j));
        list2.set(j, temp2);
    }
}

将数据次序打乱，并查看前五行内容：

In [11]:
shuffling(X, y);
for (String s : y.subList(0,5)) {
    System.out.println(s);
}

Iris-setosa
Iris-virginica
Iris-setosa
Iris-setosa
Iris-versicolor


可以看到数据前五行不再是清一色的山鸢尾花了。

下面按$1:4$的比例划分测试集和训练集：

In [12]:
ArrayList<double[]> X_train = new ArrayList<>();
ArrayList<String> y_train = new ArrayList<>();
ArrayList<double[]> X_test = new ArrayList<>();
ArrayList<String> y_test = new ArrayList<>();

for (double[] d : X.subList(0,120)) {
    X_train.add(d);
}
for (String s : y.subList(0,120)) {
    y_train.add(s);
}
for (double[] d : X.subList(120,150)) {
    X_test.add(d);
}
for (String s : y.subList(120,150)) {
    y_test.add(s);
}

In [13]:
System.out.println("训练集大小为：" + X_train.size());
System.out.println("测试集大小为：" + X_test.size());

训练集大小为：120
测试集大小为：30


可以看到，我们已经按正确的比例将数据集划分出了训练集`X_train`和`y_train`，以及测试集`X_test`和`y_test`。

<div id="3.2"></div>

## 3.2 模型评估

使用第2节构建的KNN模型对测试集进行预测，并将预测出的类别保存在ArrayList型变量`y_pred`中，这里取$k$值为3：

In [14]:
ArrayList<String> y_pred =  KNN(X_train, y_train, X_test, 3);

那么，如何知道这个模型预测能力的大小呢？正确率（accuracy）便是用来衡量模型的优劣的评估标准之一。正确率即预测正确的样本数占样本总数的比例，下面实现计算正确率的方法：

In [15]:
public void accuracy (ArrayList<String> y_test ,ArrayList<String> y_prediction){
    int count = 0;
    //如果两个列表长度不相同，则抛出异常
    if (y_test.size() != y_prediction.size()){
        throw new IllegalArgumentException("两个列表大小不一致");
    }
    //统计预测正确的个数
    for(int i = 0; i < y_test.size(); i++){
        if (y_test.get(i).equals(y_prediction.get(i))){
            count++;
        }
    }    
    //以百分比形式输出精度，并保留后两位小数
    System.out.println("预测的正确率是: " + (float)Math.round(((double)count/y_test.size()) * 10000)/100 + "%");
}

In [16]:
accuracy(y_test,y_pred);

预测的正确率是: 96.67%


由此可见，KNN模型在鸢尾花分类任务上的预测正确率可以达到96%左右！

<div id="4"></div>
# 4. 总结

本案例使用Java语言进行了一个简单的机器学习任务——对鸢尾花品种进行预测分类，在这个过程中，我们使用了`BufferedReader`读取数据，使用了数组列表`HashMap`等数据结构存储和处理数据，以及使用了Java的常用核心类的方法满足处理问题的各种需求。KNN算法的构建部分是本案例的重点，我们通过层层抽象出算法中需要处理的中间步骤，简化了KNN算法的实现过程。同时，本案例也简要介绍了机器学习的基本流程，包括数据的读取，训练集测试集的划分，模型的训练以及评估。希望读者可以在本案例的学习过程中有所收获。